<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Trees/SAT_Sudoku_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Soduku problems with SAT

The purpose of this Python notebook is to use investigate using SAT to solve Sudoku problems.  

You should have completed the notebook on SAT constructions before attempting this notebook.

You can save a local copy of this notebook in your Google account and work through it in Colab (recommended) or you can download the notebook and run it locally using Jupyter notebook or similar.

Contact me at iclimbtreesmail@gmail.com if you find any mistakes or have any suggestions.

In [1]:
# Install relevant packages
!pip install z3-solver
from z3 import *
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 19.8 MB/s eta 0:00:00


Now let's define a Sudoku problem.  We'll make a 9x9 matrix use zeros to represent unknown values.

In [ ]:
puzzle = [[0,0,9,4,2,0,0,6,0],
          [0,7,0,9,0,5,3,0,2],
          [5,0,0,0,0,3,0,9,0],
          [0,0,0,8,0,1,0,2,0],
          [2,6,0,0,0,0,0,5,1],
          [0,1,8,2,0,0,4,0,0],
          [3,8,0,0,0,4,0,1,9],
          [0,9,4,0,3,0,6,8,5],
          [0,2,1,0,0,8,0,3,0]]

In [ ]:
def expandLine(line):
    return line[0]+line[5:9].join([line[1:5]*2]*3)+line[9:13]

def drawPuzzle(puzzle):
  line0  = expandLine("╔═══╤═══╦═══╗")
  line1  = expandLine("║ . │ . ║ . ║")
  line2  = expandLine("╟───┼───╫───╢")
  line3  = expandLine("╠═══╪═══╬═══╣")
  line4  = expandLine("╚═══╧═══╩═══╝")

  symbol = " 123456789"
  nums   = [ [""]+[symbol[n] for n in row] for row in puzzle ]
  print(line0)
  for r in range(1,9+1):
      print( "".join(n+s for n,s in zip(nums[r-1],line1.split("."))) )
      print([line2,line3,line4][(r%9==0)+(r%3==0)])

In [ ]:
drawPuzzle(puzzle)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │   │ 9 ║ 4 │ 2 │   ║   │ 6 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 7 │   ║ 9 │   │ 5 ║ 3 │   │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │   ║   │   │ 3 ║   │ 9 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 8 │   │ 1 ║   │ 2 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 6 │   ║   │   │   ║   │ 5 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 1 │ 8 ║ 2 │   │   ║ 4 │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 3 │ 8 │   ║   │   │ 4 ║   │ 1 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 9 │ 4 ║   │ 3 │   ║ 6 │ 8 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 2 │ 1 ║   │   │ 8 ║   │ 3 │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝


In [ ]:
# Takes a list of z3.Bool variables and returns constraints
# ensuring that there is exactly one true
def exactly_one(x):
  return PbEq([(i,1) for i in x],1)

def solve_soduku(puzzle):
  # Create a 9 x 9 x 9 structure containing Boolean variables
  # If position (i,j,k) is true, this means position (i,j) in the puzzle matrix
  # takes color k
  x = [[[ z3.Bool("x_{%d,%d,%d}"%((i,j,k))) for k in range(0,9)] for j in range(0,9) ] for i in range(0,9) ]
  # Set up the sat solver
  s = Solver()

  # Add constraint one:  each position where the entry is already known
  # should be set to true.
  for i in range(0,9):
    for j in range(0,9):
      if puzzle[i][j] > 0:
        s.add(x[i][j][puzzle[i][j]-1] == True)


  # Add constraint two: each position in the matrix can have only one color
  # For fixed i and j, only one value of k can be true
  for i in range(0,9):
    for j in range(0,9):
      s.add(exactly_one([x[i][j][k] for k in range(0,9)]))

  # Add constraint three: each row of the table can only have each value once
  for i in range(0,9):
    for k in range(0,9):
      s.add(exactly_one([x[i][j][k] for j in range(0,9)]))

  # Add constraint four: each column of the table can only have each value once
  for j in range(0,9):
    for k in range(0,9):
      s.add(exactly_one([x[i][j][k] for i in range(0,9)]))

  # Add constraint five: each 3x3 region can only have each value once
  for i in range(0,9,3):
    for j in range(0,9,3):
      for k in range(0,9):
        s.add(exactly_one([x[i+a][j+b][k] for a in range(0,3) for b in range(0,3)]))

  # Check if it's SAT (creates the model)
  sat_result = s.check()
  print(sat_result)

  # If it isn't then return
  if sat_result == z3.sat:
    result = s.model()
    x_vals = np.array([[[int(bool(result[z3.Bool("x_{%d,%d,%d}" % (i, j, k))])) for k in range(9)] for j in range(9)] for i in range(9)] )
    solution = np.argmax(x_vals, axis=2) + 1
    drawPuzzle(solution)
  else:
    print("No solution")



In [ ]:
# Call the routine to solve the Soduku puzzle
solve_soduku(puzzle)

sat
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 1 │ 3 │ 9 ║ 4 │ 2 │ 7 ║ 5 │ 6 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 7 │ 6 ║ 9 │ 1 │ 5 ║ 3 │ 4 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │ 4 │ 2 ║ 6 │ 8 │ 3 ║ 1 │ 9 │ 7 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 4 │ 5 │ 3 ║ 8 │ 7 │ 1 ║ 9 │ 2 │ 6 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 6 │ 7 ║ 3 │ 4 │ 9 ║ 8 │ 5 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 1 │ 8 ║ 2 │ 5 │ 6 ║ 4 │ 7 │ 3 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 3 │ 8 │ 5 ║ 7 │ 6 │ 4 ║ 2 │ 1 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 9 │ 4 ║ 1 │ 3 │ 2 ║ 6 │ 8 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 2 │ 1 ║ 5 │ 9 │ 8 ║ 7 │ 3 │ 4 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝


Now let's try it with a more difficult example. Try to solve this yourself and you'll see that it's pretty hard.

In [ ]:
puzzle2 = [[0,5,0,0,0,9,0,8,0],
          [7,0,0,0,0,3,6,0,5],
          [0,0,0,0,6,0,0,1,0],
          [0,0,0,0,0,0,0,6,0],
          [0,1,0,0,3,0,4,0,9],
          [0,0,2,0,0,7,0,0,0],
          [0,9,0,0,4,0,5,0,3],
          [1,0,0,9,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,8]]
drawPuzzle(puzzle2)
solve_soduku(puzzle2)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │ 5 │   ║   │   │ 9 ║   │ 8 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │   │   ║   │   │ 3 ║ 6 │   │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │ 6 │   ║   │ 1 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │   │   ║   │ 6 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 1 │   ║   │ 3 │   ║ 4 │   │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 2 ║   │   │ 7 ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │ 9 │   ║   │ 4 │   ║ 5 │   │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║ 9 │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║   │   │ 8 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝
sat
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 6 │ 5 │ 1 ║ 7 │ 2 │ 9 ║ 3 │ 8 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 8 │ 9 ║ 4 │ 1 │ 3 ║ 6 │ 2 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │ 2 │ 4 ║ 8 │ 6 │ 5 ║ 9 │ 1 │ 7 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 9 │ 3 

Finally, let's modify the last puzzle so it is (non-obviously) impossible to solve.  I've just added a 3 to position (3,3) on the grid.  It looks innoccuous, but actually makes the puzzle unsolvable.

In [ ]:
puzzle3 = [[0,5,0,0,0,9,0,8,0],
          [7,0,0,0,0,3,6,0,5],
          [0,0,3,0,6,0,0,1,0],
          [0,0,0,0,0,0,0,6,0],
          [0,1,0,0,3,0,4,0,9],
          [0,0,2,0,0,7,0,0,0],
          [0,9,0,0,4,0,5,0,3],
          [1,0,0,9,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,8]]
drawPuzzle(puzzle3)
solve_soduku(puzzle3)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │ 5 │   ║   │   │ 9 ║   │ 8 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │   │   ║   │   │ 3 ║ 6 │   │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 6 │   ║   │ 1 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │   │   ║   │ 6 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 1 │   ║   │ 3 │   ║ 4 │   │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 2 ║   │   │ 7 ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │ 9 │   ║   │ 4 │   ║ 5 │   │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║ 9 │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║   │   │ 8 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝
unsat
No solution
